<a href="https://colab.research.google.com/github/JRKagumba/2D-video-based-exercise-classification/blob/main/00_Apply_OpenPose_to_Raw_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Drive

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Import Libraries

In [13]:
import os
import glob
import subprocess
import time

import pandas as pd
import numpy as np
import json

import shutil

### Install cmake and OpenPose dependencies

In [14]:
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

### Initialize Helper Functions

In [15]:
def convert_json2csv(json_directory, activity_folder):
    # determine the number of frames
    nframes = len(os.listdir(json_directory))

    # initialize res to be array of NaN
    res = np.zeros((nframes,75))
    res[:] = np.nan

    PROJ_JSON_ROOT = os.path.join('/content/openpose/output/')

    # read in JSON files
    for frame in range(0,nframes):
        
        test_image_json = os.path.join(PROJ_JSON_ROOT, f'{activity_folder}_{str(frame).zfill(12)}_keypoints.json')

        # if not os.path.isfile(test_image_json):
        #     break
        with open(test_image_json) as data_file:  
            data = json.load(data_file)

        for person in data['people']:
            keypoints = person['pose_keypoints_2d']
            xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
            counter = 0
            res[frame,:] = keypoints
            break

    return res

### Apply OpenPose to Raw Videos and save Time Series data

In [16]:
PROJ_SRC_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___WORKOUTS/data/tests/raw')
PROJ_SAVE_ROOT = os.path.join('/content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___WORKOUTS/data/tests/processed')

exercises_list = os.listdir(PROJ_SRC_ROOT)
run_log_data = {}

for exercise in exercises_list:
    raw_video_lst = os.listdir(os.path.join(PROJ_SRC_ROOT, exercise))

    print(exercise)
    for raw_video_file in raw_video_lst:

        src_file_path = os.path.join(PROJ_SRC_ROOT, exercise, raw_video_file)
        raw_video_folder = raw_video_file[:-4]

        print(f'\t{raw_video_folder:20}')

        #Instantiate list for each activity
        run_log_data[exercise] = []
        
        #Append file size to run logs
        file_size = os.path.getsize(src_file_path)
        run_log_data[exercise].append(file_size)

        #Clear json files from previous run
        for f in glob.glob('/content/openpose/output/*.json'):
            os.remove(f)

        #Clear avi files from previous run 
        subprocess.run(["rm", "openpose.avi"])

        #Run Pose Detection
        start = time.time()
        os.chdir('/content/openpose')
        subprocess.run(["./build/examples/openpose/openpose.bin", 
                "--video", src_file_path, 
                "--write_json", "./output/", 
                "--display", "0",  
                "--write_video", "../openpose.avi"], 
                shell=False)
        end = time.time()
        pose_detection_time = round(end-start, 1)
        run_log_data[exercise].append(pose_detection_time)  

        print(f'\t\t Pose Detected in {pose_detection_time:5}s')

        #AVI to MP4 conversion
        start = time.time()
        os.chdir('/content')
        subprocess.run(["ffmpeg", "-y", "-loglevel", "info", "-i", "openpose.avi", "output.mp4"], shell=False)
        end = time.time()
        video_convert_time = round(end-start, 1)
        run_log_data[exercise].append(video_convert_time) 

        print(f'\t\t Video Converted in {video_convert_time:5}s') 

        #Create csv output from json files
        biomech_df = pd.DataFrame(convert_json2csv("/content/openpose/output/", raw_video_folder))


        #########################################  SAVING  ##########################################

        # Make specific folder for activity files
        path = os.path.join(PROJ_SAVE_ROOT, exercise, raw_video_folder)

        if os.path.exists(path):
            pass
        else:
            os.mkdir(path)

        #### DATA
        data_save_path = os.path.join(PROJ_SAVE_ROOT, exercise, raw_video_folder, f"{raw_video_folder}.csv")
        biomech_df.to_csv(data_save_path)

        print(f'\t\t Save path : {data_save_path:20}')

        #### VIDEOS
        avi_video_src_file_path = '/content/openpose.avi'
        mp4_video_src_file_path = '/content/output.mp4'

        avi_save_path = os.path.join(PROJ_SAVE_ROOT, exercise, raw_video_folder, f"{raw_video_folder}.avi")
        mp4_save_path = os.path.join(PROJ_SAVE_ROOT, exercise, raw_video_folder, f"{raw_video_folder}.mp4")

        shutil.move(avi_video_src_file_path, avi_save_path)
        shutil.move(mp4_video_src_file_path, mp4_save_path)

        print(f'\t\t Video Saved') 


jumping_jacks_test
	01_jumping_jacks_test
		 Pose Detected in  12.3s
		 Video Converted in   2.6s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___WORKOUTS/data/tests/processed/jumping_jacks_test/01_jumping_jacks_test/01_jumping_jacks_test.csv
		 Video Saved
	02_jumping_jacks_test
		 Pose Detected in  12.7s
		 Video Converted in   2.6s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___WORKOUTS/data/tests/processed/jumping_jacks_test/02_jumping_jacks_test/02_jumping_jacks_test.csv
		 Video Saved
	03_jumping_jacks_test
		 Pose Detected in  13.5s
		 Video Converted in   3.2s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___WORKOUTS/data/tests/processed/jumping_jacks_test/03_jumping_jacks_test/03_jumping_jacks_test.csv
		 Video Saved
squats_test
	01_squats_test      
		 Pose Detected in  12.8s
		 Video Converted in   2.3s
		 Save path : /content/gdrive/MyDrive/ColabNotebooks/BiomechanicsAnalysis/___WORKOU

In [17]:
biomech_df = pd.DataFrame.from_dict(run_log_data, orient='index', columns=['File_Size', "Pose_Detection_Time(s)", "Video_Convert_Time(s)"])

biomech_df

,File_Size,Pose_Detection_Time(s),Video_Convert_Time(s)
jumping_jacks_test,2630265,13.5,3.2
squats_test,2360517,12.1,2.4
mountain_climbers_test,3592962,18.0,5.6
pushups_test,1156650,13.5,3.3
burpees_test,2899774,13.1,3.6
other_test,1919699,11.2,2.2
